In [1]:
import torch
import torch.nn.functional as F
import torch.utils.data as data_utils
import torchvision

from torch import nn

from models import BaseLine
from utils import split_indices

#### Define constants and dataset
- TODO All these definitions should be done using CL args
- Set important constant such as CUDA use and batch size

In [2]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
epochs = 50
batch_size = 32
validation_split = 0.2
dataset = torchvision.datasets.MNIST

#### Create DataSet and DataLoader objects

In [3]:
# TODO set data augmentation as described
data_transforms = {
    'train': torchvision.transforms.Compose([
        torchvision.transforms.RandomAffine(0, (0.08, 0.08)),
        torchvision.transforms.ToTensor(),
    ]),
    'test': torchvision.transforms.Compose([
        torchvision.transforms.ToTensor()
    ])
}

In [4]:
train_data = dataset('/datasets', train=True, download=False, transform=data_transforms['train'])
test_data = dataset('/datasets', train=False, download=False, transform=data_transforms['test'])

In [5]:
train_idx, val_idx = split_indices(len(train_data), validation_split)
train_sampler = data_utils.sampler.SubsetRandomSampler(train_idx)
val_sampler = data_utils.sampler.SubsetRandomSampler(val_idx)

In [6]:
train_loader = data_utils.DataLoader(train_data, batch_size=batch_size, num_workers=4, sampler=train_sampler)
val_loader  = data_utils.DataLoader(train_data, batch_size=batch_size, num_workers=4, sampler=val_sampler)
test_loader = data_utils.DataLoader(test_data, batch_size=batch_size, num_workers=4)

### TODO List
- Review data augmentation
- Implement baseline training loop
- Implement baseline validation loop

In [7]:
model = BaseLine()
model = model.to(device)

#### Optimizer definition according to default Tensorflow initiation
From Tensorflow [AdamOptimizer docs](https://www.tensorflow.org/api_docs/python/tf/train/AdamOptimizer):
```
__init__(
    learning_rate=0.001,
    beta1=0.9,
    beta2=0.999,
    epsilon=1e-08,
    use_locking=False,
    name='Adam'
)```

These are also the default values for torch.optim.Adam

In [8]:
optimizer = torch.optim.Adam(model.parameters())
criterion = nn.CrossEntropyLoss()
criterion = criterion.to(device)

In [9]:
def train(model, train_loader, epochs, loss_fn, optimizer, validation_loader=None, patience=None):
    model.train()
    loss_history = torch.zeros(epochs)
    acc_history = torch.zeros(epochs)
    best_val_acc = 0
    patience_counter = 0
    
    for epoch in range(epochs):
        loss_sum = 0
        for i, data in enumerate(train_loader):
            print('starting batch #{:5.0f}'.format(i))
            input, target = data
            input, target = input.to(device), target.to(device)
            
            optimizer.zero_grad()
            log_probs = model(input)
            
            loss = loss_fn(log_probs, target)
            loss_sum += loss.item()

            loss.backward()
            optimizer.step()

        loss_history[epoch] = loss_sum / len(train_loader)
        print('Loss in epoch {}: {}'.format(epoch+1, loss_history[epoch]))
        torch.save(model, './caps_epoch{}.pth'.format(epoch))
        if patience:
            acc_history[epoch] = evaluate_model(model, validation_loader, len(validation_loader) * validation_loader.batch_size)
            if acc_history[epoch] > best_val_acc:
                best_val_acc = acc_history[epoch]
                patience_counter = 0
            else:
                patience_counter += 1
                if patience_counter > patience:
                    print("Early Stopping in epoch {}.".format(epoch))
                    return loss_history, acc_history
        
        
    return loss_history, acc_history

In [10]:
def evaluate_model(model, data_loader, num_samples):
    hits = 0.0
    model.eval()

    for i, data in enumerate(data_loader):
        images, targets = data
        with torch.no_grad():
            images = images.to(device)
            targets = targets.to(device)
            
            log_probs = model(images)
            predictions = F.softmax(log_probs, dim=-1)
            predictions = predictions.max(dim=-1)[1]
            hits += (predictions == targets).sum().item()
        
    model.train()
    return hits/num_samples

In [11]:
loss_history, acc_history = train(model, train_loader, epochs, criterion, optimizer, val_loader, patience=10)

starting batch #    0
starting batch #    1
starting batch #    2
starting batch #    3
starting batch #    4
starting batch #    5
starting batch #    6
starting batch #    7
starting batch #    8
starting batch #    9
starting batch #   10
starting batch #   11
starting batch #   12
starting batch #   13
starting batch #   14
starting batch #   15
starting batch #   16
starting batch #   17
starting batch #   18
starting batch #   19
starting batch #   20
starting batch #   21
starting batch #   22
starting batch #   23
starting batch #   24
starting batch #   25
starting batch #   26
starting batch #   27
starting batch #   28
starting batch #   29
starting batch #   30
starting batch #   31
starting batch #   32
starting batch #   33
starting batch #   34
starting batch #   35
starting batch #   36
starting batch #   37
starting batch #   38
starting batch #   39
starting batch #   40
starting batch #   41
starting batch #   42
starting batch #   43
starting batch #   44
starting b

starting batch #  373
starting batch #  374
starting batch #  375
starting batch #  376
starting batch #  377
starting batch #  378
starting batch #  379
starting batch #  380
starting batch #  381
starting batch #  382
starting batch #  383
starting batch #  384
starting batch #  385
starting batch #  386
starting batch #  387
starting batch #  388
starting batch #  389
starting batch #  390
starting batch #  391
starting batch #  392
starting batch #  393
starting batch #  394
starting batch #  395
starting batch #  396
starting batch #  397
starting batch #  398
starting batch #  399
starting batch #  400
starting batch #  401
starting batch #  402
starting batch #  403
starting batch #  404
starting batch #  405
starting batch #  406
starting batch #  407
starting batch #  408
starting batch #  409
starting batch #  410
starting batch #  411
starting batch #  412
starting batch #  413
starting batch #  414
starting batch #  415
starting batch #  416
starting batch #  417
starting b

starting batch #  746
starting batch #  747
starting batch #  748
starting batch #  749
starting batch #  750
starting batch #  751
starting batch #  752
starting batch #  753
starting batch #  754
starting batch #  755
starting batch #  756
starting batch #  757
starting batch #  758
starting batch #  759
starting batch #  760
starting batch #  761
starting batch #  762
starting batch #  763
starting batch #  764
starting batch #  765
starting batch #  766
starting batch #  767
starting batch #  768
starting batch #  769
starting batch #  770
starting batch #  771
starting batch #  772
starting batch #  773
starting batch #  774
starting batch #  775
starting batch #  776
starting batch #  777
starting batch #  778
starting batch #  779
starting batch #  780
starting batch #  781
starting batch #  782
starting batch #  783
starting batch #  784
starting batch #  785
starting batch #  786
starting batch #  787
starting batch #  788
starting batch #  789
starting batch #  790
starting b

starting batch # 1119
starting batch # 1120
starting batch # 1121
starting batch # 1122
starting batch # 1123
starting batch # 1124
starting batch # 1125
starting batch # 1126
starting batch # 1127
starting batch # 1128
starting batch # 1129
starting batch # 1130
starting batch # 1131
starting batch # 1132
starting batch # 1133
starting batch # 1134
starting batch # 1135
starting batch # 1136
starting batch # 1137
starting batch # 1138
starting batch # 1139
starting batch # 1140
starting batch # 1141
starting batch # 1142
starting batch # 1143
starting batch # 1144
starting batch # 1145
starting batch # 1146
starting batch # 1147
starting batch # 1148
starting batch # 1149
starting batch # 1150
starting batch # 1151
starting batch # 1152
starting batch # 1153
starting batch # 1154
starting batch # 1155
starting batch # 1156
starting batch # 1157
starting batch # 1158
starting batch # 1159
starting batch # 1160
starting batch # 1161
starting batch # 1162
starting batch # 1163
starting b

starting batch # 1492
starting batch # 1493
starting batch # 1494
starting batch # 1495
starting batch # 1496
starting batch # 1497
starting batch # 1498
starting batch # 1499
Loss in epoch 1: 0.38858702778816223
starting batch #    0
starting batch #    1
starting batch #    2
starting batch #    3
starting batch #    4
starting batch #    5
starting batch #    6
starting batch #    7
starting batch #    8
starting batch #    9
starting batch #   10
starting batch #   11
starting batch #   12
starting batch #   13
starting batch #   14
starting batch #   15
starting batch #   16
starting batch #   17
starting batch #   18
starting batch #   19
starting batch #   20
starting batch #   21
starting batch #   22
starting batch #   23
starting batch #   24
starting batch #   25
starting batch #   26
starting batch #   27
starting batch #   28
starting batch #   29
starting batch #   30
starting batch #   31
starting batch #   32
starting batch #   33
starting batch #   34
starting batch # 

...

starting batch # 1192
starting batch # 1193
starting batch # 1194
starting batch # 1195
starting batch # 1196
starting batch # 1197
starting batch # 1198
starting batch # 1199
starting batch # 1200
starting batch # 1201
starting batch # 1202
starting batch # 1203
starting batch # 1204
starting batch # 1205
starting batch # 1206
starting batch # 1207
starting batch # 1208
starting batch # 1209
starting batch # 1210
starting batch # 1211
starting batch # 1212
starting batch # 1213
starting batch # 1214
starting batch # 1215
starting batch # 1216
starting batch # 1217
starting batch # 1218
starting batch # 1219
starting batch # 1220
starting batch # 1221
starting batch # 1222
starting batch # 1223
starting batch # 1224
starting batch # 1225
starting batch # 1226
starting batch # 1227
starting batch # 1228
starting batch # 1229
starting batch # 1230
starting batch # 1231
starting batch # 1232
starting batch # 1233
starting batch # 1234
starting batch # 1235
starting batch # 1236
starting b

starting batch #   63
starting batch #   64
starting batch #   65
starting batch #   66
starting batch #   67
starting batch #   68
starting batch #   69
starting batch #   70
starting batch #   71
starting batch #   72
starting batch #   73
starting batch #   74
starting batch #   75
starting batch #   76
starting batch #   77
starting batch #   78
starting batch #   79
starting batch #   80
starting batch #   81
starting batch #   82
starting batch #   83
starting batch #   84
starting batch #   85
starting batch #   86
starting batch #   87
starting batch #   88
starting batch #   89
starting batch #   90
starting batch #   91
starting batch #   92
starting batch #   93
starting batch #   94
starting batch #   95
starting batch #   96
starting batch #   97
starting batch #   98
starting batch #   99
starting batch #  100
starting batch #  101
starting batch #  102
starting batch #  103
starting batch #  104
starting batch #  105
starting batch #  106
starting batch #  107
starting b

starting batch #  436
starting batch #  437
starting batch #  438
starting batch #  439
starting batch #  440
starting batch #  441
starting batch #  442
starting batch #  443
starting batch #  444
starting batch #  445
starting batch #  446
starting batch #  447
starting batch #  448
starting batch #  449
starting batch #  450
starting batch #  451
starting batch #  452
starting batch #  453
starting batch #  454
starting batch #  455
starting batch #  456
starting batch #  457
starting batch #  458
starting batch #  459
starting batch #  460
starting batch #  461
starting batch #  462
starting batch #  463
starting batch #  464
starting batch #  465
starting batch #  466
starting batch #  467
starting batch #  468
starting batch #  469
starting batch #  470
starting batch #  471
starting batch #  472
starting batch #  473
starting batch #  474
starting batch #  475
starting batch #  476
starting batch #  477
starting batch #  478
starting batch #  479
starting batch #  480
starting b

starting batch #  809
starting batch #  810
starting batch #  811
starting batch #  812
starting batch #  813
starting batch #  814
starting batch #  815
starting batch #  816
starting batch #  817
starting batch #  818
starting batch #  819
starting batch #  820
starting batch #  821
starting batch #  822
starting batch #  823
starting batch #  824
starting batch #  825
starting batch #  826
starting batch #  827
starting batch #  828
starting batch #  829
starting batch #  830
starting batch #  831
starting batch #  832
starting batch #  833
starting batch #  834
starting batch #  835
starting batch #  836
starting batch #  837
starting batch #  838
starting batch #  839
starting batch #  840
starting batch #  841
starting batch #  842
starting batch #  843
starting batch #  844
starting batch #  845
starting batch #  846
starting batch #  847
starting batch #  848
starting batch #  849
starting batch #  850
starting batch #  851
starting batch #  852
starting batch #  853
starting b

starting batch # 1182
starting batch # 1183
starting batch # 1184
starting batch # 1185
starting batch # 1186
starting batch # 1187
starting batch # 1188
starting batch # 1189
starting batch # 1190
starting batch # 1191
starting batch # 1192
starting batch # 1193
starting batch # 1194
starting batch # 1195
starting batch # 1196
starting batch # 1197
starting batch # 1198
starting batch # 1199
starting batch # 1200
starting batch # 1201
starting batch # 1202
starting batch # 1203
starting batch # 1204
starting batch # 1205
starting batch # 1206
starting batch # 1207
starting batch # 1208
starting batch # 1209
starting batch # 1210
starting batch # 1211
starting batch # 1212
starting batch # 1213
starting batch # 1214
starting batch # 1215
starting batch # 1216
starting batch # 1217
starting batch # 1218
starting batch # 1219
starting batch # 1220
starting batch # 1221
starting batch # 1222
starting batch # 1223
starting batch # 1224
starting batch # 1225
starting batch # 1226
starting b

In [12]:
evals = evaluate_model(model, test_loader, len(test_data))

In [13]:
evals

0.9927

In [14]:
print(evals*100)

99.27
